# Project Plan

After discussing with Robin Burke, the general model will include have multiple levels for detecting for the recommender system
- Last FM to start the cluster
- use eucliedean distance, and only draw connections between "djable" nodes

## Data to Test

- [🎹 Spotify Tracks Dataset](https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset)

In [597]:
## IMPORTS ##
import seaborn as sns
import pandas as pd
import networkx as nx
import numpy as np
import requests
import re
from collections import Counter
import os
from kmodes.kmodes import KModes
import spotipy

In [598]:
# looking at dataset of songs
df = pd.read_csv('dataset.csv')

In [599]:
# All the genres in the dataframe
np.unique(df.track_genre)

array(['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient',
       'anime', 'black-metal', 'bluegrass', 'blues', 'brazil',
       'breakbeat', 'british', 'cantopop', 'chicago-house', 'children',
       'chill', 'classical', 'club', 'comedy', 'country', 'dance',
       'dancehall', 'death-metal', 'deep-house', 'detroit-techno',
       'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm',
       'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk',
       'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove',
       'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle',
       'heavy-metal', 'hip-hop', 'honky-tonk', 'house', 'idm', 'indian',
       'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol',
       'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino',
       'malay', 'mandopop', 'metal', 'metalcore', 'minimal-techno', 'mpb',
       'new-age', 'opera', 'pagode', 'party', 'piano', 'pop', 'pop-film',
       'pow

[keys.py](https://github.com/schwartzadev/dj-recommender/blob/master/keys.py)


- Rules for Building Edges
    - BPM within 8 of each other
    - must be within similar key --> Camelot
    - must be at least danceability of 0.5, otherwise add and ignore
    - must be at least popularity of 0.7, otherwise add and ignore

In [489]:
# Helper Functions

def _calc_bpm_range(bpm,brange=6):
    '''
    returns list of bpm range
    '''
    return [bpm-brange,bpm+brange]

def song_id_search(_id):
    return df[df['track_id']==_id]

def find_neighbors(song,danceability=0.5,popularity=50):
    brange = _calc_bpm_range(song.tempo)
    small_df = df[(df['tempo']>brange[0]) & (df['tempo']<brange[1]) & (df['track_genre'] == song.track_genre) & (df['danceability']>danceability) & (df['popularity']>popularity) & (df['key'] == song.key)]
    return small_df['track_id'].tolist() 

In [603]:
bieb = df.iloc[30001]

In [604]:
bieb.track_name

'Let Me Love You'

In [605]:
# we can build edges using filtering this way
find_neighbors(bieb)

['7BKLCZ1jbUBVqRi2FVlTVw',
 '0lYBSQXN6rCTvUZvg9S0lU',
 '4w8niZpiMy6qz1mntFA5uM',
 '0KoiMHhqKLoL4xM54rcWYY',
 '5ZAUiOlYURVJEJ5ktV03v5',
 '1YMBg7rOjxzbya0fPOYfNX',
 '2FqkTu4FhwDWn9hzEaWWCE',
 '0U6bQIAh6MCGo1xjbIIx2S',
 '5nihLVpUd4birzwDlSp1Ph',
 '3RgQFleuuQKcCmig9ItyxQ',
 '5masKPHeAOVNgxdLebIcK7',
 '0e8fTfa0ZKCwOi2UdULooT',
 '6oJ6le65B3SEqPwMRNXWjY']

In [606]:
# there are duplicates, group track_genre's together? Would that even make sense?
test = song_id_search('2FqkTu4FhwDWn9hzEaWWCE')
test

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
30403,30403,2FqkTu4FhwDWn9hzEaWWCE,The Chainsmokers,So Far So Good,I Love U,69,185522,False,0.651,0.719,...,-5.804,1,0.0318,0.143,0.000036,0.0948,0.81,103.981,4,edm
31455,31455,2FqkTu4FhwDWn9hzEaWWCE,The Chainsmokers,So Far So Good,I Love U,69,185522,False,0.651,0.719,...,-5.804,1,0.0318,0.143,0.000036,0.0948,0.81,103.981,4,electro
53355,53355,2FqkTu4FhwDWn9hzEaWWCE,The Chainsmokers,So Far So Good,I Love U,69,185522,False,0.651,0.719,...,-5.804,1,0.0318,0.143,0.000036,0.0948,0.81,103.981,4,house


In [607]:
# checking to see how many duplicates there are
len(df)-len(np.unique(df.track_id))

24259

In [608]:
# 25k duplicates?!?!! Let's drop those

In [609]:
# Drop unneed columns
df = df.drop(columns=[
    'Unnamed: 0', 
    'explicit',
    'loudness',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness']
            )


In [610]:
cols = list(df.columns)
cols.remove('track_genre')
print(cols)

['track_id', 'artists', 'album_name', 'track_name', 'popularity', 'duration_ms', 'danceability', 'energy', 'key', 'mode', 'valence', 'tempo', 'time_signature']


In [611]:
df = df.groupby(cols)['track_genre'].apply(','.join).reset_index()

In [612]:
df

,track_id,artists,album_name,track_name,popularity,duration_ms,danceability,energy,key,mode,valence,tempo,time_signature,track_genre
0,0000vdREvCVMxbQTkS888c,Rill,Lolly,Lolly,44,160725,0.910,0.37400,8,0,0.432,104.042,4,german
1,000CC8EParg64OmTxVnZ0p,Glee Cast,Glee Love Songs,It's All Coming Back To Me Now (Glee Cast Vers...,47,322933,0.269,0.51600,0,1,0.341,178.174,4,club
2,000Iz0K615UepwSJ5z2RE5,Paul Kalkbrenner;Pig&Dan,X,Böxig Leise - Pig & Dan Remix,22,515360,0.686,0.56000,5,0,0.108,119.997,4,minimal-techno
3,000RDCYioLteXcutOjeweY,Jordan Sandhu,Teeje Week,Teeje Week,62,190203,0.679,0.77000,0,1,0.839,161.721,4,hip-hop
4,000qpdoc97IMTBvF8gwcpy,Paul Kalkbrenner,Zeit,Tief,19,331240,0.519,0.43100,6,0,0.234,129.971,4,minimal-techno
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90455,7zxHiMmVLt4LGWpOMqOpUh,Haricharan;Gopi Sundar,Bangalore Days,"Aethu Kari Raavilum - From ""Bangalore Days""",56,325156,0.766,0.38200,7,0,0.672,119.992,4,pop-film
90456,7zxpdh3EqMq2JCkOI0EqcG,Piano Genie,Disney Favourites,"Two Worlds (From ""Tarzan"")",23,109573,0.529,0.00879,10,1,0.510,82.694,4,disney
90457,7zyYmIdjqqiX6kLryb7QBx,Eric Chou,學著愛,以後別做朋友,61,260573,0.423,0.36000,3,1,0.291,130.576,4,mandopop
90458,7zybSU9tFO9HNlwmGF7stc,Stereoclip,Echoes,Sunset Drive,54,234300,0.649,0.83400,10,0,0.150,125.004,4,electronic


In [613]:
# Lol, why are there still duplicates
len(df)-len(np.unique(df.track_id))

720

In [614]:
# list of unique ids
unique = np.unique(df.track_id)

# all ids
ids = df.track_id

In [615]:
# duplicates left
len(ids) - len(unique)

720

In [616]:
id_count = dict(Counter(ids))

In [617]:
dup_list = [key for key,val in id_count.items() if val > 1]

In [618]:
# list of duplicates captured
len(dup_list)

720

In [623]:
# list of duplicates
# dup_list

In [620]:
# slight difference in popularity, hm
song_id_search('00YwP3wJWiG8IxAA7OS9lo')

,track_id,artists,album_name,track_name,popularity,duration_ms,danceability,energy,key,mode,valence,tempo,time_signature,track_genre
109,00YwP3wJWiG8IxAA7OS9lo,Anupam Roy,Doorbiney Chokh Rakhbona,Amake Amar Moto Thakte Dao,46,319946,0.566,0.419,7,1,0.186,147.881,4,"singer-songwriter,songwriter"
110,00YwP3wJWiG8IxAA7OS9lo,Anupam Roy,Doorbiney Chokh Rakhbona,Amake Amar Moto Thakte Dao,47,319946,0.566,0.419,7,1,0.186,147.881,4,"indian,indie-pop,indie,k-pop"


In [621]:
# another difference in popularity
song_id_search('014SIjoLDG1Ku19c5FlDYh')

,track_id,artists,album_name,track_name,popularity,duration_ms,danceability,energy,key,mode,valence,tempo,time_signature,track_genre
189,014SIjoLDG1Ku19c5FlDYh,Creedence Clearwater Revival,Pumpkin Patch Hits,I Put A Spell On You,0,271786,0.393,0.732,4,0,0.621,100.41,4,country
190,014SIjoLDG1Ku19c5FlDYh,Creedence Clearwater Revival,Pumpkin Patch Hits,I Put A Spell On You,3,271786,0.393,0.732,4,0,0.621,100.41,4,rock


In [622]:
# ANOTHER difference in popularity, weird
song_id_search('0DqLuhTD1xI8mb2gY5YoLM')

,track_id,artists,album_name,track_name,popularity,duration_ms,danceability,energy,key,mode,valence,tempo,time_signature,track_genre
2547,0DqLuhTD1xI8mb2gY5YoLM,Håkan Hellström,Det är så jag säger det,Den fulaste flickan i världen,35,197826,0.292,0.812,9,1,0.502,108.782,4,swedish
2548,0DqLuhTD1xI8mb2gY5YoLM,Håkan Hellström,Det är så jag säger det,Den fulaste flickan i världen,36,197826,0.292,0.812,9,1,0.502,108.782,4,goth


At this point I am thinking of dropping the rows that are duplicates, and removing the one with higher popularity. I don't think it will be the end of the world, but because this data is weird, I don't want to recommend it probably.

In [629]:
df = df[~df['track_id'].isin(dup_list)]

In [632]:
# hell yeah, duplicate values are gone
df[df['track_id']=='00YwP3wJWiG8IxAA7OS9lo']

,track_id,artists,album_name,track_name,popularity,duration_ms,danceability,energy,key,mode,valence,tempo,time_signature,track_genre


### Sudo Code

I want to create a song object so that I can pass the data in a box to the visualization in D3. I'm hoping I can project the image and add the spotify link to the visualization.

1. Create Song Objects with attributes
    - Attributes
        - Song ID
        - Name
        - Artist
        - Spotify Link
        - spotify genre
        - popularity
        - key (convert to camelot)
        - tempo
        - lyrics
        - duration
        - explicit
    - Methods
        - Generate Spotify Link
        - get last fm track tag
        - from Song_ID (class method)
        - get lyrics
        - get valid tempo range
        - get neighbors (using filtering)
2. Go through entire track list
    - for a song in the song list
        - build a list of node list songs that share similar features
3. Playlist Object
    - Attributes
        - Song Objects
        - BPM range
        - Key Range
        - Genre
    - Methods
        - Create in Spotify(using Spotipy)
        - From 2 songs (class method)
            - short
            - different paths
        - Add (add to graph, use graph logic to create new playlist?)
4. SongGraph Object
   - Attributes
       - Last Update
   - Methods
       - Save to GraphML
       - From GraphML
       - Visualize
       - _add_song
       - _remove_song

Simple, just referenced when Playlist Object is implemented, contains metadata about last update, etc. It's nice because it will also have ways to save the current graph to graphml, etc.
 

In [593]:
class Song:
    def __init__(self, song_id, name, artists, popularity, key, tempo, duration, explicit=None,lyrics=None):
        self.song_id = song_id
        self.name = name
        self.artists = artists
        self.popularity = popularity
        self.key = key
        self.tempo = tempo
        self._duration = duration

    @classmethod
    def from_df_row(cls,df_row):
        return Song(df_row.track_id, df_row.track_name, df_row.artists, df_row.popularity, df_row.key, df_row.tempo, df_row.duration_ms)        

    @classmethod
    def from_spot_id(cls,_id):
        pass

    @property
    def spot_link(self):
        return f'https://open.spotify.com/track/{self.song_id}'
    
    @property
    def duration(self):
        _sec=int((self._duration/1000)%60)
        _min=int((self._duration/(1000*60))%60)
        return f'{_min}min {_sec}sec'
    
    @property
    def valid_bpm_range(self):
        pass
    
    @property
    def stats(self):
        output = f'''Song ID: {self.song_id}
Song Name: {self.name}
Song Artists: {self.artists}
Song Key: {self.key}
Song Tempo: {self.tempo}
Song Duration {self.duration}'''
        print(output)
    
    def __str__(self):
        return f'Song Object: \'{self.name} by {self.artists}\''
    
    def __repr__(self):
        return f'ID: {self.song_id} | Name: {self.name} | Song Artists: {self.artists} | Song Key: {self.key} | Song Tempo: {self.tempo} | Song Duration {self.duration}'
    


In [ ]:
# I could do it by removing all of the rows that have id's in the dup_list

In [600]:
# Creating Object

first = df.iloc[0]

song = Song.from_df_row(first)

In [601]:
song.duration

'3min 50sec'